# JOIN SCHOOLS FROM ALL TERMS 

In [1]:
import pandas as pd
import re

In [2]:
all_years = [2018,2019,2020,2021]

In [3]:
all_schools = pd.read_csv(f'clean_schools/clean_schools_2017_year.csv')
all_schools['ANIO']=2017

for year in all_years:
        print(year)
        term_data = pd.read_csv(f'clean_schools/clean_schools_{year}_year.csv')
        term_data['ANIO'] = year
        all_schools = pd.concat([all_schools, term_data])

2018
2019
2020
2021


In [4]:
def normalize_name(txt1):
    if(pd.isna(txt1)):
        return ""
    clean_dict={
        'COL ': 'COLEGIO ',
        'FUND ': 'FUNDACION '        
    }
    clean_name = txt1
    for abr in clean_dict.keys():
        clean_name = clean_name.replace(abr, clean_dict[abr])
    clean_name = clean_name.strip()
    return clean_name

In [10]:
schools_id = all_schools[['COLE_COD_DANE_SEDE', 'COLE_JORNADA']].drop_duplicates()
count_all = len(schools_id)
dane = 'COLE_COD_DANE_SEDE'
shift = 'COLE_JORNADA'
school_txt_fields = ['COLE_NOMBRE_SEDE', 'COLE_MCPIO_UBICACION', 'COLE_DEPTO_UBICACION',  'COLE_GENERO', 
'COLE_CALENDARIO', 'COLE_BILINGUE', 'COLE_CARACTER', 'COLE_SEDE_PRINCIPAL',
'COLE_AREA_UBICACION']
school_num_fields = ['COLE_COD_MCPIO_UBICACION', 'COLE_COD_DEPTO_UBICACION']
score_fields = [ field for field in all_schools.columns if re.search('mean',field)]
clean_schools = []
all_warnings = []
for idx, row in schools_id.iterrows():
    print(f"{idx}/{count_all}", end= '\r')
    clean_school = {}
    id_dane = row['COLE_COD_DANE_SEDE']
    school_shift = row['COLE_JORNADA']
    clean_school['COLE_COD_DANE_SEDE']= id_dane 
    clean_school['COLE_JORNADA'] = school_shift
    school_year = all_schools[(all_schools[shift] == row[shift]) & (all_schools[dane] == row[dane])]
    for field in school_txt_fields: 
        field_value = school_year[field].drop_duplicates()
        field_value = set([ normalize_name(i) for i in field_value])
        if(len(field_value)>1):
            last_register = school_year.loc[school_year['ANIO'] == school_year['ANIO'].max(),field]
            clean_school[field] = last_register.values[0]
            all_warnings.append({
                'school_id': id_dane,
                'field': field,
                'amount': len(field_value),
                'values': field_value,
                'kept': last_register.values
            });
        else: 
            clean_school[field] = field_value.pop()

    for field in school_num_fields: 
        field_value = school_year[field].drop_duplicates()
        if(len(field_value)>1):
            print(f'WARNING! School seat with DANE id {id_dane} has {len(field_value)} {field}, wich are: \n {field_value.values}')
        if(len(field_value) == 0):
            print(f'WARNING! School seat with DANE id {id_dane} has no values for field: {field}')

        clean_school[field] = field_value.values[0]
    
    for year in school_year['ANIO'] :
        for field in score_fields: 
            key_val = f'{field.capitalize()}_{year}'
            clean_school[key_val] = school_year.loc[school_year['ANIO']==year, field].values[0]
    
    
    clean_schools.append(clean_school)

In [7]:
joined_schools = pd.DataFrame(clean_schools)
joined_schools.to_csv(f"clean_schools/clean_schools_joined.csv")

In [9]:
joined_schools.head(3)

,COLE_COD_DANE_SEDE,COLE_JORNADA,COLE_NOMBRE_SEDE,COLE_MCPIO_UBICACION,COLE_DEPTO_UBICACION,COLE_GENERO,COLE_CALENDARIO,COLE_BILINGUE,COLE_CARACTER,COLE_SEDE_PRINCIPAL,...,Punt_c_naturales_mean_2020,Punt_sociales_ciudadanas_mean_2020,Punt_ingles_mean_2020,Punt_global_mean_2020,Punt_lectura_critica_mean_2021,Punt_matematicas_mean_2021,Punt_c_naturales_mean_2021,Punt_sociales_ciudadanas_mean_2021,Punt_ingles_mean_2021,Punt_global_mean_2021
0,319001003379,MAÑANA,LICEO BELLO HORIZONTE,[POPAYÁN],CAUCA,MIXTO,A,N,ACADÉMICO,S,...,[50.0],[51.46153846153846],[50.26923076923077],[262.61538461538464],[53.333333333333336],[51.53846153846154],[51.23076923076923],[48.66666666666666],[51.41025641025641],[256.02564102564105]
1,325123000125,MAÑANA,COLEGIO PARROQUIAL INTEGD SANTA CRUZ,CACHIPAY,CUNDINAMARCA,MIXTO,A,N,ACADÉMICO,S,...,[54.31818181818182],[52.5],[51.54545454545455],[271.5],[52.1764705882353],[53.47058823529412],[49.1764705882353],[49.05882352941177],[52.11764705882353],[255.41176470588235]
2,468867000170,COMPLETA,INSTITUTO CRISTIANO DE PROMOCION CAMPESINA ICPROC,VETAS,SANTANDER,MIXTO,A,N,ACADÉMICO,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
